In [13]:
%matplotlib notebook

# Nutils by example

### About Nutils
Nutils is a python based finite element package. Nutils does not come with a GUI. You have to write a python script yourself. Nutils provides a set of tools help you implement a weak formulation quite easily.

Some basic understanding of python is useful. If you are unfamiliar with python you might want to browser through a tutorial.

### Einstein notation
All mathematical equations in this document are written using the Einstein Notation. All indices of vectors, matrices or higher-dimensional arrays are explicitly listed. The following special cases apply:

* Indices following a comma denote derivatives:

$$ u_{i,jk} $$

is equivalient to 

$$ \frac{\partial^2 u_i}{\partial x_j \partial x_k}.$$

* Repeated indices in a term are summed:

$$ u_{,ii} + v_j w_{,j} $$

is equivalent to 

$$ \sum_i \frac{\partial^2 u_i}{\partial^2 x_i} + \sum_j v_j \frac{\partial w}{\partial x_j}$$

## Laplace's equation with Neumann BC's

Consider the following system ([Laplace's equation](https://en.wikipedia.org/wiki/Laplace%27s_equation) with homogeneous Neumann boundary conditions):

$$ \left\{ \begin{array}{ll} - u_{,kk} = 0 & \text{in } \Omega \\ u_{,k}n_k = 0 & \text{on } \partial \Omega \end{array}\right. $$

with $n$ the outward normal and domain $\Omega=[-1/\sqrt{2}, 1/\sqrt{2}]^2$. Note that this system is singular.

Let $\varphi : \Omega \rightarrow \mathbb{R}^N$ a basis on domain $\Omega$. A weak formulation of this system reads:

> Find $w_j$ such that for all $i\in \{0,1,...,N-1\}$    $$ \int_\Omega \varphi_{i,k} \varphi_{j,k} w_j d\Omega=0$$

The solution $u$ is then approximated by $\varphi_j w_j$, i.e. the dot product of the basis $\varphi$ with vector $w$.

Let $T$ be a structured partition of $8\times 8$ equally sized elements of $\Omega$ and $\varphi$ a linear basis on 
$T$. The following Nutils script implements the weak formulation and finds a solution:



In [40]:
import numpy
from nutils import *

# construct topology, geometry and basis
verts = numpy.linspace(-0.5**0.5, 0.5**0.5, 9)
domain, geom = mesh.rectilinear([verts, verts])

# construct integrand
ns = function.Namespace()
ns.x = geom
ns.basis = domain.basis('spline', degree=1)

# construct matrix
A = domain.integrate(
    ns.eval_ij('basis_i,k basis_j,k'),
    geometry=geom, ischeme='gauss3')

# solve linear system
w = A.solve()

solve > right hand side is zero


### Importing nutils

In this script the line

In [41]:
from nutils import *

loads the python package `nutils`. This line is mandatory for all nutils scripts. See the [python docs](https://docs.python.org/3/tutorial/modules.html#packages) for more information on importing packages.

### Topology and geometry
With the line

In [42]:
domain, geom = mesh.rectilinear([verts, verts])

we create a structured two-dimensional mesh. The argument `[verts, verts]` of function `mesh.rectilinear` specifies the vertices of the mesh per dimension, i.e. `verts` in the first dimension and also `verts` in the second dimension. `mesh.rectilinear([verts])` would create a one-dimensional mesh and `mesh.rectilinear([verts, verts, verts])` a three-dimensional structured mesh.

The variable `verts` contains a list of vertices, e.g. `[0, 0.25, 0.5, 0.75, 1]`. The function numpy.linspace creates a list of equally spaced numbers between two end points, e.g. `numpy.linspace(0, 1, 5)` returns `[0, 0.25, 0.5 0.75, 1]`. The first two arguments describe the two end points, the third argument the number of points. The number of elements corresponding to a list of vertices is always the number of vertices minus one.

The function `mesh.rectilinear` returns a topology and a geometry object, in the example respectively stored in variables domain and geom. The 

topology object contains information about the elements and the connectivity between the elements, but no geometric information. The geometry object is a Nutils function that connects a topological coordinate to a geometric coordinate. To illustrate the difference the following example generates the same one-dimensional topology twice, but with different geometry functions:

In [43]:
verts1 = [0, 1, 2, 3, 4]
domain1, geom1 = mesh.rectilinear([verts1])
verts2 = [0, 0.25, 0.5, 0.75, 1]
domain2, geom2 = mesh.rectilinear([verts2])

In [44]:
basis = domain.basis('spline', degree=1)
grad = function.grad(basis, geom)
A = domain.integrate(
    function.outer(grad,grad),
    geometry=geom, ischeme='gauss3')

### Basis and namespace
with

In [45]:
ns = function.Namespace()
ns.x = geom
ns.basis = domain.basis('spline', degree=1)

a linear basis is created on topology domain and stored in variable basis. The first argument to `domain.basis` specifies the type of the basis, e.g. `'spline'` for a spline basis or `'discont'` for a basis suitable for discontinuous Galerking FEM. The second argument specifies the degree of the basis.

The Nutils function basis, like geom, can be 'evaluated' on the topology, and returns a vector of length $N^2$, where $N$ is the number of basis functions. In the example script there are 91 basis functions, one per vertex if the basis is linear. The function attribute shape returns the shape of a function. For example `ns.basis.shape` returns (91,) and `geom.shape` (2,).

In [46]:
print(ns.basis.shape)
print(geom.shape)

(81,)
(2,)


### Numerical integration

Every topology object has an `integrate` method that can integrate an integrand (first argument) using a numerical scheme specified by the [keyword argument](https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments) `ischeme` on a certain geometry specified by the keyword argument geometry. The following python code evaluates the integral

$$ A_ij = \int_\Omega \varphi_{i,k}\varphi_{j,k} d\Omega :$$

In [47]:
A = domain.integrate(
    ns.eval_ij('basis_i,k basis_j,k'),
    geometry=geom, ischeme='gauss3')

The `integrate` method generates a sparse matrix if the integrand has two dimensions, i.e. two indices. An integrand with only one index will result in a vector.

The shape of the sparse matrix A is equal to the shape of the integrand. In this example `A.shape` equals `(basis['i,k'] * basis['j,k']).shape` equals `(91, 91)`.

In [48]:
A.shape

(81, 81)

### Matrices

A sparse matrix has a `solve` method that solves linear problems of the form $A x = b$, where $A$ is the sparse matrix and $b$ is the right hand side. Passing the right hand side as the first argument to solve, e.g. `A.solve(b)` yields the solution $x$. By default a sparse direct solver is used to solve the linear system. If the keyword argument `tol` is supplied, e.g. `A.solve(b, tol=1e-8)` the linear system is solved using GMRES. If the keyword argument `symmetric=True` is supplied, the linear system is solved using CG. Note that it is your responsibility to ensure the matrix is symmetric positive definite. If the keyword argument `precon='spilu'` is supplied the iterative solver is preconditioned with ILU. By omitting the first (positional) argument the right hand side is assumed to be zero.

Instead of using Nutils' sparse matrix class, you can extract a [scipy sparse](http://docs.scipy.org/doc/scipy-0.14.0/reference/sparse.html) matrix object by calling the matrix method `toscipy()`:

In [49]:
B = A.toscipy()
type(B)

scipy.sparse.csr.csr_matrix

## Laplace's equation with Dirichlet BC's

We replace the homogeneous Neumann boundary conditions with Dirichlet boundary conditions:


$$ \left\{ \begin{array}{ll} - u_{,kk} = 0 & \text{in } \Omega \\ u = f & \text{on } \partial \Omega \end{array}\right. $$

with $f$ given by

$$ f(x) = sin(x_0) exp(x_1) $$

This system is not singular. Note that the solution to this problem is $u=f$. 

A weak formulation of this system reads

> Find $w_j$ such that for all $i\in \{0,1,...,N-1\}$ for which $\varphi_i$ as *no* support on the boundary $\partial \Omega$ 
> $$ \int_\Omega \varphi_{i,k} \varphi_{j,k} w_j d\Omega=0$$ 
> and such that for all $i\in \{0,1,...,N-1\}$ for which $\varphi_i$ *has* support on the boundary $\partial \Omega$
> $$\int_{\partial \Omega}\varphi_i\left(\varphi_j w_j - f\right) d\Omega = 0.$$

The solution $u$ is again approximated by $\varphi_jw_j$. Note that the equation above is an $L_2-$ projection of $f$ onto basis $\varphi_i$ limited to the boudnary $\partial \Omega$.

Again, let $T$ be a structured partition of $8\times 8$ equally sized elements of $\Omega$ and $\varphi$ a linear basis on $T$. The following Nutils script implements the weak formulation and finds a solution:

In [50]:
import numpy
from nutils import *

# construct topology, geometry and basis
verts = numpy.linspace(-0.5**0.5, 0.5**0.5, 9)
domain, geom = mesh.rectilinear([verts, verts])

# construct integrand
ns = function.Namespace()
ns.x = geom
ns.basis = domain.basis('spline', degree=1)

# construct matrix
A = domain.integrate(
    ns.eval_ij('basis_i,k basis_j,k'),
    geometry=geom, ischeme='gauss3')

x0,x1 = geom
f = function.sin(x0) * function.exp(x1)

# construct dirichlet boundary constraints
cons = domain.boundary.project(f, onto=ns.basis, geometry=geom, ischeme='gauss3')

# solve linear system
w = A.solve(constrain=cons)

project > solve > solving system using sparse direct solver
project > constrained 32/81 dofs, error 2.75e-04/area
solve > solving system using sparse direct solver


### Dirichlet constraints

We explain the differences with the previous script. We create a function `f` by calling `function.sin` on the first component of the two-dimensional geometry and `function.exp` on the second component:

In [51]:
x0, x1 = geom
f = function.sin(x0) * function.exp(x1)

The statement `x0, x1 = geom` unpacks the two components of the function `geom`.

The dirichlet constraint is applied in the following two statements:

In [52]:
cons = domain.boundary.project(
    f, onto=basis, geometry=geom, ischeme='gauss3')
w = A.solve(constrain=cons)

project > solve > solving system using sparse direct solver
project > constrained 32/81 dofs, error 2.75e-04/area
solve > solving system using sparse direct solver


actually another topology object, which supports the integrate and project methods. The result vector cons contains values for all basis functions that have support on domain.boundary, otherwise float('nan').

The method solve of a Nutils matrix supports an additional keyword argument constrain. If supplied, all values, i.e. those that are not float('nan'), are copied to the solution vector and the remainder is solved.

To illustrate what happens, consider the linear system $A_{ij}x_j = b_j$ and let $i_U$ be an index belonging to the unconstrained set and $i_C$ to the constrained set and let $c_{j_C}$ be the vector of constraints. Let $\tilde{x}$ be the solution to the constrained problem. Then we have $\tilde{x}_{j_C} = c_{j_C}$ and

$$ A_{iUjU} \tilde{x}_{jU} = b_{iU} - A_{iUjC}c_{j_C}.$$

Note that $A_{ij}\tilde{x}_j \neq b_j$!

### Nutils' log and running from the commandline

** Note: ** The following code will not work when running nutils interactively and is intended for command-line implementations of the solver

Nutils creates html log files of your simulations. By default the logs are stored in the folder `public_html` in your home directory. The file `log.html` in this directory points to the latest simulation.

The following script models the same problem as discussed above, but supports log files:



In [ ]:
import numpy
from nutils import *

def main(nelems=8, degree=1):

    # construct topology, geometry and basis
    verts = numpy.linspace(-0.5**0.5, 0.5**0.5, nelems+1)
    domain, geom = mesh.rectilinear([verts, verts])
    ischeme = 'gauss3'

    # construct integrand
    ns = function.Namespace()
    ns.x = geom
    ns.basis = domain.basis('spline', degree=1)

    # construct matrix
    A = domain.integrate(
        ns.eval_ij('basis_i,k basis_j,k'),
        geometry=geom, ischeme=ischeme)

    x0, x1 = geom
    f = function.sin(x0) * function.exp(x1)

    # construct dirichlet boundary constraints
    cons = domain.boundary.project(
        f, onto=ns.basis, geometry=geom, ischeme=ischeme)

    # solve linear system
    w = A.solve(constrain=cons)

    # construct solution
    u = ns.basis.dot(w)

    # plot
    points, colors = domain.elem_eval(
        [geom, u], ischeme='bezier3', separate=True)
    with plot.PyPlot('solution') as plt:
        plt.mesh(points, colors)
        plt.colorbar()
        plt.show()

if __name__ == '__main__': # run from commandline
    cli.run(main)

The solution:

![The solution](https://joostvanzwieten.github.io/nutils-by-example/logs/laplace_dirichlet.py/solution000.png)

We explain the differences with the previous script. All statements of the previous script excluding the `import` statement are put into the function main:

In [ ]:
def main(nelems=8, degree=1):

    ... # contents previous script plus plotting

if __name__ == '__main__':
    util.run(main)

You can run this script by typing

```
python3 laplace_dirichlet.py
```
in a shell. The arguments `nelms` and `degree` can be specified on the command line, e.g.
```
python3 laplace_dirichlet.py --nelems=16 --degree=2
```
runs the script with variables `nelms` set to `16` and `degree` to `2`

The statement

In [69]:
u = ns.basis.dot(w)

creates a solution function, the equivalent of $u=\varphi_iw_i$ where $\varphi_i$ is the basis.

The function `domain.elem_eval` evaluates Nutils functions specified via the first (positional) argument at several points on the topology and the function `plt.mesh` plots the results, where the first argument defines a position and the second a colour value.

## Laplace's equation with mixed BC's
We partially replace the Dirichlet boundary conditions with Neumann boundary conditions:
